In [30]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.wkt import loads

sys.path.insert(0, "../src")

import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Local/input files
iso3 = 'KHM'
out_folder = "c:/WBG/Work/KHM_Energy/data"
wsf_file = os.path.join(out_folder, "WSF", "wsf.tif")
ghsl_file = os.path.join(out_folder, "GHSL", "ghsl.tif")
overture_buildings = os.path.join(out_folder, "overture", "overture_download_2024_03_29.csv")
overture_raster = os.path.join(out_folder, "overture", "overture_download_2024_03_29.tif")
overture_raster_points = os.path.join(out_folder, "overture", "overture_download_2024_03_29_points.tif")
for file in [wsf_file, ghsl_file]:
    if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))

# get country extent from geopandas
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
country = world[world.iso_a3 == iso3]

C:\Users\WB411133\AppData\Local\Temp\ipykernel_25180\3039392467.py:14: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world_filepath = gpd.datasets.get_path('naturalearth_lowres')


In [6]:
""" Not working with World Bank Firewall
# Download the WSF data
if not os.path.exists(wsf_file):
    print("Downloading WSF data")
    wsf_data, wsf_profile = dMisc.download_WSF(country, out_file = wsf_file)

wsf_r = rasterio.open(wsf_file)
"""
print("FUBAR")

FUBAR


In [7]:
#Clip GHSL using local files
local_version = r"J:\Data\GLOBAL\GHSL\Built\GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0.tif"
if not os.path.exists(ghsl_file):
    ghsl_raster = rasterio.open(local_version)
    data, profile = rMisc.clipRaster(ghsl_raster, country)
    with rasterio.open(ghsl_file, 'w', **profile) as dst:
        dst.write(data)
ghsl_r = rasterio.open(ghsl_file)

In [8]:
# read in and process Overture buildings
ob = pd.read_csv(overture_buildings)
ob_geoms = ob['wkt'].apply(loads)
inB = gpd.GeoDataFrame(ob, geometry=ob_geoms, crs=4326)
inB.head()


,class,height,wkt,geometry
0,NaN,NaN,"POLYGON ((103.3991092 13.6076154, 103.3991148 ...","POLYGON ((103.39911 13.60762, 103.39911 13.607..."
1,NaN,NaN,"POLYGON ((103.819971 13.2832912, 103.8199974 1...","POLYGON ((103.81997 13.28329, 103.82000 13.283..."
2,NaN,NaN,"POLYGON ((103.8152555 13.2890315, 103.8151626 ...","POLYGON ((103.81526 13.28903, 103.81516 13.289..."
3,NaN,NaN,"POLYGON ((105.5873344 12.3655821, 105.5873972 ...","POLYGON ((105.58733 12.36558, 105.58740 12.365..."
4,NaN,NaN,"POLYGON ((103.0758412 13.2660819, 103.0758304 ...","POLYGON ((103.07584 13.26608, 103.07583 13.266..."


In [9]:
# attempt to rasterrize the buildings as polygons
if not os.path.exists(overture_raster):
    rasterized_buildings = rMisc.rasterizeDataFrame(inB, templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(overture_raster, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
overture_r = rasterio.open(overture_raster)

In [10]:
# attempt to rasterrize the buildings as points
if not os.path.exists(overture_raster_points):
    inB_points = inB.copy()
    inB_points['geometry'] = inB_points['geometry'].centroid
    rasterized_buildings = rMisc.rasterizeDataFrame(inB_points, templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(overture_raster_points, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
overture_r_points = rasterio.open(overture_raster_points) 

In [32]:
# Compare rasterized buildings with built area dataset
### Open both datasets and threshold them to get built area data
o_thresh = 1
ghsl_thresh = 3000

o_data = overture_r_points.read(1)
o_data = (o_data > o_thresh).astype('uint8')

ghsl_data = ghsl_r.read(1)
ghsl_data = (ghsl_data > ghsl_thresh).astype('uint8') * 10

combo_data = o_data + ghsl_data

# Write out the combined data
out_file = os.path.join(out_folder, "overture_vs_ghsl.tif")
if not os.path.exists(out_file):
    meta = overture_r_points.meta.copy()
    meta.update(dtype=rasterio.uint8, nodata=0)
    with rasterio.open(out_file, 'w', **meta) as out_raster:
        out_raster.write_band(1, combo_data)
